In [1]:
import pandas as pd
import numpy as np

dft = pd.read_csv("times_computed.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)
dfd = pd.read_csv("complete_distances.csv", delimiter=";", encoding = "ISO-8859-1", index_col=0)

In [2]:
def get_row(df, string):
    return pd.Series(df.index).str.contains(string, case=False)
def get_column(df, string):
    return pd.Series(df.columns).str.contains(string, case=False)

In [3]:
table=pd.read_csv("route.csv", delimiter=";")
time_table=pd.read_csv("route_time.csv", delimiter=";")

In [4]:
import datetime
def difference(day, index, index2):
    time = datetime.datetime.strptime(time_table[day][index], '%H:%M')
    time2 = datetime.datetime.strptime(time_table[day][index2], '%H:%M')
    return ((time2-time).seconds)/60

In [17]:
"""gained_time1=0
to_2=[]
days = ['martedi', 'mercoledi','giovedì']
#for day in days:
day='martedi'
for i in range(len(time_table[day].dropna())-2):
    try:
        diff=difference(day, i, i+1)
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][i+1]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time1 :
            gained_time1+=(diff-ctime)
            print(f"{day}: primo : {paese} : {gained_time1}")
        else:
            gained_time1+=diff
            print(f"{day}: failed : {ctime} : {diff} : {gained_time1}")
            to_2.append(i)
    except:
        print(value)
    #if diff<dft[row[paese].values]:
    #    print("Not acceptable!")"""

'gained_time1=0\nto_2=[]\ndays = [\'martedi\', \'mercoledi\',\'giovedì\']\n#for day in days:\nday=\'martedi\'\nfor i in range(len(time_table[day].dropna())-2):\n    try:\n        diff=difference(day, i, i+1)\n        row=dft[get_row(dft, table[day][i]).values]\n        paese=table[day][i+1]\n        value=row.loc[:,get_column(dft, paese).values]\n        ctime=float(value.values[0][0])\n        #gained_time1=gained_time1+(diff-float(value.values[0][0]))\n        #gained_time2=gained_time2+(diff-float(value.values[0][0]))\n        if (diff+(diff/20))>ctime-gained_time1 :\n            gained_time1+=(diff-ctime)\n            print(f"{day}: primo : {paese} : {gained_time1}")\n        else:\n            gained_time1+=diff\n            print(f"{day}: failed : {ctime} : {diff} : {gained_time1}")\n            to_2.append(i)\n    except:\n        print(value)\n    #if diff<dft[row[paese].values]:\n    #    print("Not acceptable!")'

In [18]:
"""impossible=[]
gained_time2=0
for i in range(len(to_2)-1):
    try:
        curr=to_2[i]
        diff=difference(day, curr, to_2[i+1])
        row=dft[get_row(dft, table[day][i]).values]
        paese=table[day][to_2[i+1]]
        value=row.loc[:,get_column(dft, paese).values]
        ctime=float(value.values[0][0])
        #gained_time1=gained_time1+(diff-float(value.values[0][0]))
        #gained_time2=gained_time2+(diff-float(value.values[0][0]))
        if (diff+(diff/20))>ctime-gained_time2 :
            gained_time2+=(diff-ctime)
            print(f"{day}: secondo : {paese} : {gained_time2}")
        else:
            print(f"{day}: impossible : {paese} : {diff}  {ctime}")
            impossible.append(i)
    except:
        print(f"-----{value}")"""

'impossible=[]\ngained_time2=0\nfor i in range(len(to_2)-1):\n    try:\n        curr=to_2[i]\n        diff=difference(day, curr, to_2[i+1])\n        row=dft[get_row(dft, table[day][i]).values]\n        paese=table[day][to_2[i+1]]\n        value=row.loc[:,get_column(dft, paese).values]\n        ctime=float(value.values[0][0])\n        #gained_time1=gained_time1+(diff-float(value.values[0][0]))\n        #gained_time2=gained_time2+(diff-float(value.values[0][0]))\n        if (diff+(diff/20))>ctime-gained_time2 :\n            gained_time2+=(diff-ctime)\n            print(f"{day}: secondo : {paese} : {gained_time2}")\n        else:\n            print(f"{day}: impossible : {paese} : {diff}  {ctime}")\n            impossible.append(i)\n    except:\n        print(f"-----{value}")'

In [40]:
#FINAL ALGORITHM
start = dft[get_row(dft, 'Cremona, sesto 39').values]
day='giovedi'
#Liste dei due furgoni
f1 = []
f2 = []
#indici
i1 = -1
i2 = -1
#Le righe delle distanze del corrente paese in cui si trovano i furgoni, inizialmente lo stesso
row1 = start
row2 = start
#Lista dei paesi non raggiungibili
impossibile = []
#
for i in range(len(time_table[day].dropna())):
    paese=table[day][i]
    #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
    value1=row1.loc[:,get_column(dft, paese).values]
    value2=row2.loc[:,get_column(dft, paese).values]
    #Salvataggio del tempo utile per raggiungere la destinazione in base alla tabella degli orari
    if(i1 != -1 and i2 != -1):
        diff1=difference(day, i1, i)
        diff2=difference(day, i2, i)
        #print(f"{diff1}, {diff2}")
    else:
        #Per la prima tratta l'indice non ha valore utile, si ottiene la differenza staticamente
        if(i1 == -1):
            diff1 = value1.values[0][0]
        if(i2 == -1):
            diff2 = value2.values[0][0]
            
    if(diff1 == 0):
        f2.append(i)
        i2 = i
        row2 = dft[get_row(dft, table[day][i]).values]
        continue
    if(diff2 == 0):
        f1.append(i)
        i1 = i
        row1 = dft[get_row(dft, table[day][i]).values]
        continue
        
    #Se uno dei due furgoni può rispettare l'orario (con tolleranza 5 minuti) si sceglie il migliore
  
    if(float(value1.values[0][0])<= float(diff1) + 15 or float(value2.values[0][0])<= float(diff2) + 15):
        if(value1.values[0][0] <= value2.values[0][0]):
            f1.append(i)
            i1 = i
            """print(i)
            if(i >= 13):
                print(f"{diff1}, {diff2}")"""
            row1 = dft[get_row(dft, table[day][i]).values]
        else:
            f2.append(i)
            i2 = i
            row2 = dft[get_row(dft, table[day][i]).values]
    #Impossibilità di raggiungere la locazione in tempo
    else:
        print(f"current time1 {diff1} time {float(value1.values[0][0])} , current time2 {diff2} time {float(value2.values[0][0])}")
        impossibile.append(i)
    
"""print(f1)
print(f2)"""
print(table[day][impossibile])
print(table[day][f1])
print(table[day][f2])

Series([], Name: giovedi, dtype: object)
0               manerbio
1         Cremona, sesto
2          pizzighettone
3       casalpusterlengo
6          pizzighettone
9                  izano
11             offanengo
13    Crema, Civerchi, 9
18         casalmaggiore
Name: giovedi, dtype: object
4     Cremona, Ugolani Dati, 4
5     Cremona, Ugolani Dati, 4
7                    grontardo
8                    pescarolo
10                   vescovato
12                      cingia
14                    solarolo
15                     ostiano
16                  martignana
17              persico dosimo
19                     gadesco
20                     piadena
21              isola dovarese
Name: giovedi, dtype: object


In [27]:
#FINAL ALGORITHM
start = dft[get_row(dft, 'Cremona, sesto 39').values]
day='venerdi'
#Liste dei due furgoni
f1 = []

#indici
i1 = -1

#Le righe delle distanze del corrente paese in cui si trovano i furgoni, inizialmente lo stesso
row1 = start

#Lista dei paesi non raggiungibili
impossibile = []
for i in range(len(time_table[day].dropna())-1):
    paese=table[day][i]
    print(paese)
    #Distanze in tempo dal paese corrente di ognuno dei furgoni alla destinazione
    value1=row1.loc[:,get_column(dft, paese).values]

    #Salvataggio del tempo utile per raggiungere la destinazione in base alla tabella degli orari
    if(i1 != -1):
        diff1=difference(day, i1, i)
        #print(f"{diff1}, {diff2}")
    else:
        #Per la prima tratta l'indice non ha valore utile, si ottiene la differenza staticamente
        if(i1 == -1):
            diff1 = value1.values[0][0]
        
    #Se uno dei due furgoni può rispettare l'orario (con tolleranza 5 minuti) si sceglie il migliore
  
    if(float(value1.values[0][0])<= float(diff1)):
        
        f1.append(i)
        i1 = i
        row1 = dft[get_row(dft, table[day][i]).values]
       
    #Impossibilità di raggiungere la locazione in tempo
    else:
        print(f"current time1 {diff1} time {float(value1.values[0][0])}")
        impossibile.append(i)
    
print(f1)
print(impossibile)
table[day][impossibile]

Capergnanica
monte cremasco
spino d'adda
pandino
bagnolo
crema, civerchi
vaiano
trescore
rivolta d'adda
dovera
agnadello
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[]


Series([], Name: venerdi, dtype: object)

In [57]:
dtime=pd.read_csv("sospirot.csv", delimiter=";", encoding = "ISO-8859-1")
for i in range(len(dtime)):
    for j in range(len(dtime.columns)):
        curr=dtime.iloc[i].iloc[j]
        dtime.iat[i,j]=round(float(curr)/1000)
dtime

,Sospiro
0,61.316667
1,29.150000
2,52.450000
3,6.933333
4,55.116667
...,...
60,63.083333
61,12.916667
62,15.900000
63,8.166667


In [13]:
#time_
table[day][10:13]

10    Offanengo
11      Piadena
12      Gussola
Name: martedi, dtype: object